In [1]:
# Initial imports.
import pandas as pd
import numpy as np
from numpy import random
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Import RDS password
from config import rds_pwd

import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.9 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-05-25 04:02:37--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-05-25 04:02:37 (11.8 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Wine-Project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Configure settings for RDS
mode = "append"
#jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
jdbc_url="jdbc:postgresql://wine-final-project.czqkltznl3rl.us-east-2.rds.amazonaws.com/winedb"
#config = {"user":"postgres",
          #"password": "<password>",
          #"driver":"org.postgresql.Driver"}
config = {"user":"wineuser",
          "password": rds_pwd,
          "driver":"org.postgresql.Driver"}

In [5]:
#select_sql = "(SELECT points, variety, avg_temp, avg_airp, avg_humid, avg_precip FROM machine"
wine_rds = spark.read.jdbc(url=jdbc_url, table='weather_mapscaled', properties=config)
#county_df = spark.read.jdbc(url=jdbc_url, table='counties', properties=config)
wine_rds.show(8)

+--------------------+---+----+--------------------+--------------------+--------------------+--------------------+-------+------+----------+-------------+
|  winery_search_term|lat| lng|            avg_temp|            avg_airp|           avg_humid|          avg_precip|country|points|   variety|       winery|
+--------------------+---+----+--------------------+--------------------+--------------------+--------------------+-------+------+----------+-------------+
|   LaZarre winery US| 36|-121|279.6850000000000...|1018.230000000000...|73.42000000000000...|0.025000000000000000|     US|    88|Chardonnay|      LaZarre|
|Negretti winery I...| 45|   8|275.6025000000000...|1016.337500000000...|79.04000000000000...|0.070000000000000000|  Italy|    88|Chardonnay|     Negretti|
|Pico Maccario win...| 45|   8|280.6950000000000...|1017.880000000000...|70.50250000000000...|0.147500000000000000|  Italy|    85|Chardonnay|Pico Maccario|
|Farnese winery Italy| 43|  12|278.1475000000000...|1014.4450000

In [6]:
wine_rds.columns

['winery_search_term',
 'lat',
 'lng',
 'avg_temp',
 'avg_airp',
 'avg_humid',
 'avg_precip',
 'country',
 'points',
 'variety',
 'winery']

In [7]:
winescaled = wine_rds.select('lat','lng','avg_temp','avg_airp','avg_humid','avg_precip','variety','points', (wine_rds.points >= 90).alias('success'))
winescaled.show(8)

+---+----+--------------------+--------------------+--------------------+--------------------+----------+------+-------+
|lat| lng|            avg_temp|            avg_airp|           avg_humid|          avg_precip|   variety|points|success|
+---+----+--------------------+--------------------+--------------------+--------------------+----------+------+-------+
| 36|-121|279.6850000000000...|1018.230000000000...|73.42000000000000...|0.025000000000000000|Chardonnay|    88|  false|
| 45|   8|275.6025000000000...|1016.337500000000...|79.04000000000000...|0.070000000000000000|Chardonnay|    88|  false|
| 45|   8|280.6950000000000...|1017.880000000000...|70.50250000000000...|0.147500000000000000|Chardonnay|    85|  false|
| 43|  12|278.1475000000000...|1014.445000000000...|81.86750000000000...|0.182500000000000000|Chardonnay|    85|  false|
| 43|  12|278.1475000000000...|1014.445000000000...|81.86750000000000...|0.182500000000000000|Chardonnay|    86|  false|
| 43|  12|278.1475000000000...|1

In [8]:
allwines = pd.DataFrame(winescaled.select('lat',
 'lng',
 'avg_temp',
 'avg_airp',
 'avg_humid',
 'avg_precip','success','variety').collect(), columns=['lat',
 'lng',
 'avg_temp',
 'avg_airp',
 'avg_humid',
 'avg_precip','success','variety'], dtype=float)
allwines.dtypes

lat           float64
lng           float64
avg_temp      float64
avg_airp      float64
avg_humid     float64
avg_precip    float64
success       float64
variety        object
dtype: object

In [9]:
county_weath = spark.read.jdbc(url=jdbc_url, table='uscounty', properties=config)
county_weath.show(8)

+----+---+---+----------+--------+-------------+-----------+--------------------+--------------------+--------------------+--------------------+
| zip|lat|lng|      city|state_id|   state_name|county_name|            avg_temp|            avg_airp|           avg_humid|          avg_precip|
+----+---+---+----------+--------+-------------+-----------+--------------------+--------------------+--------------------+--------------------+
|1001| 42|-73|    Agawam|      MA|Massachusetts|    Hampden|270.8050000000000...|1016.122500000000...|72.21500000000000...|0.125000000000000000|
|1002| 42|-72|   Amherst|      MA|Massachusetts|  Hampshire|270.8050000000000...|1016.122500000000...|72.21500000000000...|0.125000000000000000|
|1005| 42|-72|     Barre|      MA|Massachusetts|  Worcester|270.7250000000000...|1014.960000000000...|71.75250000000000...|0.135000000000000000|
|1029| 42|-73| East Otis|      MA|Massachusetts|  Berkshire|270.8050000000000...|1016.122500000000...|72.21500000000000...|0.12500

In [107]:
county_weath.select('lat','lng')

DataFrame[lat: int, lng: int]

In [10]:
county_weath.columns

['zip',
 'lat',
 'lng',
 'city',
 'state_id',
 'state_name',
 'county_name',
 'avg_temp',
 'avg_airp',
 'avg_humid',
 'avg_precip']

In [11]:
county_ids = pd.DataFrame((county_weath.select('zip',
                                             'city',
                                             'state_id',
                                             'state_name',
                                             'county_name').collect()), columns=['zip',
                                             'city',
                                             'state_id',
                                             'state_name',
                                             'county_name']).dropna()
county_ids.head(4)

,zip,city,state_id,state_name,county_name
0,1001,Agawam,MA,Massachusetts,Hampden
1,1002,Amherst,MA,Massachusetts,Hampshire
2,1005,Barre,MA,Massachusetts,Worcester
3,1029,East Otis,MA,Massachusetts,Berkshire


In [12]:
bigcounty = pd.DataFrame(county_weath.select('zip',
 'lat',
 'lng',
 'avg_temp',
 'avg_airp',
 'avg_humid',
 'avg_precip').collect(),columns=['zip',
 'lat',
 'lng',
 'avg_temp',
 'avg_airp',
 'avg_humid',
 'avg_precip'],dtype=float).dropna()
bigcounty.head(5)

,zip,lat,lng,avg_temp,avg_airp,avg_humid,avg_precip
0,1001.0,42.0,-73.0,270.805,1016.1225,72.2150,0.125
1,1002.0,42.0,-72.0,270.805,1016.1225,72.2150,0.125
2,1005.0,42.0,-72.0,270.725,1014.9600,71.7525,0.135
3,1029.0,42.0,-73.0,270.805,1016.1225,72.2150,0.125
4,1054.0,42.0,-72.0,270.805,1016.1225,72.2150,0.125


In [13]:
bigcounty.dtypes

zip           float64
lat           float64
lng           float64
avg_temp      float64
avg_airp      float64
avg_humid     float64
avg_precip    float64
dtype: object

In [14]:
countydrops = ['zip']

In [15]:
mer_df = allwines[allwines['variety'] == 'Merlot']
syr_df = allwines[allwines['variety'] == 'Syrah']
savyb_df = allwines[allwines['variety'] == 'Sauvignon Blanc']
ries_df = allwines[allwines['variety'] == 'Riesling']
cabsav_df = allwines[allwines['variety'] == 'Cabernet Sauvignon']
char_df = allwines[allwines['variety'] == 'Chardonnay']
pino_df = allwines[allwines['variety'] == 'Pinot Noir']

In [66]:
winedfs = [mer_df,syr_df,savyb_df,ries_df,cabsav_df,char_df,pino_df]
for x in winedfs:
  print(len(x))

2974
3928
4718
5105
8812
11298
12641


In [16]:
mer_df.dtypes

lat           float64
lng           float64
avg_temp      float64
avg_airp      float64
avg_humid     float64
avg_precip    float64
success       float64
variety        object
dtype: object

In [17]:
mer_df.columns

Index(['lat', 'lng', 'avg_temp', 'avg_airp', 'avg_humid', 'avg_precip',
       'success', 'variety'],
      dtype='object')

In [73]:
winedropcols = ['variety','success']

In [74]:
#features and targets
variety = mer_df.copy()
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99, bootstrap=False, max_samples=500)
rf.fit(X, y)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=500,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [75]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.7741935483870968

In [76]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
mer_preds = rf.predict(Xcount)

In [77]:
#features and targets
variety = syr_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99, bootstrap=False, max_samples=500)
rf.fit(X, y)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=500,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [78]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.6629327902240326

In [79]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
syr_preds = rf.predict(Xcount)

In [80]:
#features and targets
variety = savyb_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99, bootstrap=False, max_samples=500)
rf.fit(X, y)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=500,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [81]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.7542372881355932

In [82]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
savyb_preds = rf.predict(Xcount)

In [83]:
#features and targets
variety = ries_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99, bootstrap=False, max_samples=500)
rf.fit(X, y)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=500,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [84]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.716523101018011

In [85]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
ries_preds = rf.predict(Xcount)

In [86]:
#features and targets
variety = cabsav_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99, bootstrap=False, max_samples=500)
rf.fit(X, y)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=500,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [87]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.7058556513844757

In [88]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
cabsav_preds = rf.predict(Xcount)

In [89]:
#features and targets
variety = char_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99, bootstrap=False, max_samples=500)
rf.fit(X, y)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=500,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [90]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.703716814159292

In [91]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
char_preds = rf.predict(Xcount)

In [92]:
#features and targets
variety = pino_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99, bootstrap=False, max_samples=500)
rf.fit(X, y)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=500,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [93]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.6368237899398924

In [94]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
pino_preds = rf.predict(Xcount)

In [95]:
counties_preds = bigcounty.copy()
counties_preds['mer_success'] = mer_preds
counties_preds['syr_success'] = syr_preds
counties_preds['savyb_success'] = savyb_preds
counties_preds['ries_success'] = ries_preds
counties_preds['cabsav_success'] = cabsav_preds
counties_preds['char_success'] = char_preds
counties_preds['pinot_success'] = pino_preds

In [96]:
counties_preds.describe()

,zip,lat,lng,avg_temp,avg_airp,avg_humid,avg_precip,mer_success,syr_success,savyb_success,ries_success,cabsav_success,char_success,pinot_success
count,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000
mean,50734.467967,38.357103,-91.311421,274.303437,1017.654591,74.131337,0.106546,0.025627,0.153760,0.080780,0.039554,0.144847,0.088022,0.146518
std,24597.722869,5.101211,12.403000,6.620178,4.407807,6.428987,0.098159,0.158063,0.360819,0.272573,0.194964,0.352044,0.283406,0.353723
min,1001.000000,26.000000,-124.000000,260.640000,962.630000,37.947500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30127.000000,34.000000,-99.000000,269.471250,1016.560000,71.022500,0.017500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,50515.000000,38.000000,-89.000000,274.070000,1019.040000,75.952500,0.097500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,71632.500000,42.000000,-82.000000,279.122500,1020.105000,78.457500,0.155000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,99401.000000,49.000000,-68.000000,295.845000,1022.525000,87.015000,0.917500,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [97]:
counties_preds['pinot_success'].value_counts()

0.0    1532
1.0     263
Name: pinot_success, dtype: int64

In [98]:
#counties_preds.to_csv('county_predictions.csv')

In [99]:
#spark_counties = spark.createDataFrame(counties_preds)
#spark_counties.show()

In [100]:
#spark_counties.write.jdbc(url=jdbc_url, table='county_predictions', mode=mode, properties=config)

In [101]:
#sparked_preds = spark.read.jdbc(url=jdbc_url, table='county_predictions', properties=config)

In [102]:
#sparked_preds.show()

In [103]:
#winners_df = pd.DataFrame(sparked_preds.select('zip','lat','lng','state_id','county_name','mer_success','rose_success','syr_success','savyb_success','ries_success','cabsav_success','char_success','pinot_success').collect(), columns=['zip','lat','lng','state','county_name','mer_success','rose_success','syr_success','savyb_success','ries_success','cabsav_success','char_success','pinot_success'], dtype=float)
#winners_df.describe()

In [104]:
#winners_df.columns

In [105]:
#winners_df

In [106]:
#winners_df['mer_success'].value_counts()